# MORPC Insights - Housing Price and Income Profile

## Overview

The Multiple Listing Service (MLS) is a database used by realtors to track and share real estate sales.  MORPC has access to MLS data for the Central Ohio area thanks to the generosity of Columbus Realtors.  Each record in the database represents a listing of a property for sale and contains attributes that track the status and the end result of the sale, including sale price. 

The American Community Survey produces estimates of median household income for geographies down to the tract level.

This notebook produces a tileset that includes a summary of MLS median home sale price and median household income for the MORPC 15-county region and the counties and communities therein.  This notebook is the final stage in a pipeline that fetches, standardizes, and summarizes the data from several sources.

## Setup

### Load required libraries

In [ ]:
import pandas as pd
import frictionless
import os
import sys
import math
import datetime
import textwrap
import matplotlib
from matplotlib import pyplot as plt
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### User-specified parameters

In [ ]:
YEAR_RANGE = [2012, 2023]

# Suppress sales data for geograpy/year combinations in which there were fewer than this many sales
LOW_COUNT_SUPPRESSION_THRESHOLD = 10

# Suppress median income data for geography/year combinations in which the relative margin of error is greater than this ratio
MEDIAN_INCOME_SUPPRESSION_THRESHOLD = 0.25

# Suppress mean income data for geography/year combinations in which the relative margin of error is greater than this ratio
MEAN_INCOME_SUPPRESSION_THRESHOLD = 0.25

### Static parameters

In [ ]:
INPUT_DIR = os.path.normpath("./input_data")

OUTPUT_DIR = os.path.normpath("./output_data")

CHART_DIRNAME = "charts"

# Price to income ratio values that designate how affordability the market is
# Values represent the floor for the specified affordability level
# See https://www.jchs.harvard.edu/blog/home-price-income-ratio-reaches-record-high-0
AFFORDABILITY_BREAKPOINTS = {
    # "Affordable": 0,
    "Moderately Unaffordable":3,
    "Seriously Unaffordable":4,
    "Severely Unaffordable":5
}

### Define inputs

#### Create input data directory

Create input data directory if it doesn't exist.

In [ ]:
inputDir = os.path.normpath(INPUT_DIR)
if not os.path.exists(inputDir):
    os.makedirs(inputDir)

#### Housing cost and income profile

In [ ]:
HOUSING_PROFILE_RESOURCE = "../morpc-housingcost-profile/output_data/morpc-housingcost-profile.resource.yaml"
print("Resource: {}".format(HOUSING_PROFILE_RESOURCE))

#### Geography lookup table [375]

In [ ]:
GEOS_LOOKUP_TABLE_RESOURCE = os.path.normpath("../morpc-geos-collect/output_data/morpc-geos-lookup.resource.yaml")
print("Resource file: {}".format(GEOS_LOOKUP_TABLE_RESOURCE))

#### MORPC member list [122]

In [ ]:
MEMBERS_DATA_PATH = "../morpc-lookup/Member_List.xlsx"
MEMBERS_SHEET = "Current Year Members"
MEMBERS_SCHEMA_PATH = "../morpc-lookup/Member_List_schema.json"
print("Data: {}, sheet '{}'".format(MEMBERS_DATA_PATH, MEMBERS_SHEET))
print("Schema: {}".format(MEMBERS_SCHEMA_PATH))

### Define outputs

#### Create output data directory

Create output data directory if it doesn't exist.

In [ ]:
outputDir = os.path.normpath(OUTPUT_DIR)
if not os.path.exists(outputDir):
    os.makedirs(outputDir)   

In [ ]:
chartDir = os.path.join(outputDir, CHART_DIRNAME)
if not os.path.exists(chartDir):
    os.makedirs(chartDir)    

#### Housing cost and income profile (Insights version)

In [ ]:
HOUSING_COST_FILENAME = "housingcost-long.csv"
HOUSING_COST_TABLE_PATH = os.path.join(outputDir, HOUSING_COST_FILENAME)
HOUSING_COST_SCHEMA_PATH = HOUSING_COST_TABLE_PATH.replace(".csv",".schema.yaml")
HOUSING_COST_RESOURCE_PATH = HOUSING_COST_TABLE_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(HOUSING_COST_TABLE_PATH))
print("Schema: {}".format(HOUSING_COST_SCHEMA_PATH))
print("Resource file: {}".format(HOUSING_COST_RESOURCE_PATH))

## Prepare input data

### Load geography lookup table

In [ ]:
(geosRaw, geosRawResource, geosRawSchema) = morpc.frictionless_load_data(GEOS_LOOKUP_TABLE_RESOURCE, validate=True, archiveDir=inputDir)

In [ ]:
geosRaw.head()

In [ ]:
geos = geosRaw.copy()

### Load summarized housing cost profile from upstream workflows

In [ ]:
(housingRaw, housingRawResource, housingRawSchema) = morpc.frictionless_load_data(HOUSING_PROFILE_RESOURCE, validate=True, archiveDir=inputDir)

In [ ]:
housingRaw.head()

In [ ]:
housing = housingRaw.copy()

### MORPC member list

The output data will include all communities for whom data has not been suppressed, however we will only show data for MORPC members in the platform.

Load the member table.

In [ ]:
membersRaw = pd.read_excel(MEMBERS_DATA_PATH, sheet_name=MEMBERS_SHEET)
membersRaw.head()

Load the schema.

In [ ]:
membersSchema = morpc.load_avro_schema(MEMBERS_SCHEMA_PATH)

Verify that the fields are all the expected types.

In [ ]:
members = morpc.cast_field_types(membersRaw, membersSchema)

Extract only the communities which are themselves a member.

In [ ]:
members = members.loc[members["Local Member"] == True].copy()

The records in the member table are all county parts.  For places we need to subsitute the GEOID for the full place rather than the county part.

In [ ]:
members["PLACEFP"] = members["GEOID"].apply(lambda x:x[11:16])
members["COUSUBFP"] = members["GEOID"].apply(lambda x:x[14:19])
members["COUNTYID"] = members["County"].map(morpc.CONST_COUNTY_NAME_TO_ID)
members["GEOIDFQ"] = None
temp = members.loc[members["GovType"] == "Township"].copy()
temp["GEOIDFQ"] = temp["GEOID"]
members.update(temp)
temp = members.loc[members["GovType"] != "Township"].copy()
temp["GEOIDFQ"] = "1600000US39" + temp["PLACEFP"]
members.update(temp)

Now extract just the list of member GEOIDs.  The steps above likely produced duplicate records for places, so extract only the unique GEOIDs.

In [ ]:
memberList = list(members["GEOIDFQ"].unique())

We also need to append the list of counties.  We will include data for all counties regardless of membership status.

In [ ]:
memberList += ["0500000US{}".format(morpc.CONST_COUNTY_NAME_TO_ID[x]) for x in morpc.CONST_REGIONS['REGION15']]

Finally we need to append the ID for the MORPC region.

In [ ]:
memberList.append("M010000US001")

## Transform data to format required by Insights platform

Create a working copy.

In [ ]:
housing = housingRaw.copy()

Extract records from the specified year range.

In [ ]:
housing = housing.loc[housingRaw["YEAR"].isin(range(YEAR_RANGE[0], YEAR_RANGE[1]+1))].copy()

Convert the SUMLEVEL code to a text description of the geography type.

In [ ]:
housing["GEOTYPE"] = housing["SUMLEVEL"].map(morpc.HIERARCHY_STRING_LOOKUP)

Join some fields from the geography lookup table, dropping the existing name field to make way for the name field in the join table.  Add the state FIPS code to the county FIPS code and use the resulting GEOID to look up the county name.

In [ ]:
housing = housing.drop(columns="NAME").merge(geos[["GEOIDFQ","COUNTYFP","NAME"]], on="GEOIDFQ")
housing["COUNTYFP"] = "39" + housing["COUNTYFP"]
housing["COUNTY"] = housing["COUNTYFP"].map(morpc.CONST_COUNTY_ID_TO_NAME)

Extract the unincorporated townships (SUMLEVEL 070) and append a suffix that includes "Township" and the county name to disambiguate townships from places and counties.

In [ ]:
temp = housing.loc[housing["SUMLEVEL"] == "070"].copy()
temp["NAME"] = temp["NAME"] + " Township (" + temp["COUNTY"] + ")"
housing.update(temp, overwrite=True, errors="ignore")

Extract the counties (SUMLEVEL 050) and append a "County" suffix to disambiguate the counties from places.

In [ ]:
temp = housing.loc[housing["SUMLEVEL"] == "050"].copy()
temp["NAME"] = temp["NAME"] + " County"
housing.update(temp, overwrite=True, errors="ignore")

Suppress records where the number of sales is lower than the specified threshold.

In [ ]:
lowCountSuppressionIndex = housing.loc[housing["Number of sales"] < LOW_COUNT_SUPPRESSION_THRESHOLD].index
housing.loc[lowCountSuppressionIndex, [
    'Median home sale price (2023 dollars)',
    'Median home sale price (year-of dollars)',
    'Median sale price/income ratio (MSA med inc)',
    'Median sale price/income ratio (geography med inc)']] = None
print("Suppressed {} records where number of sales was below the suppression threshold".format(len(lowCountSuppressionIndex)))

Suppress records where the median income margin of error is more than the specified threshold.

In [ ]:
medianIncomeSuppressionIndex = housing.loc[(housing["Median income MOE (year-of dollars)"]/housing["Median income (year-of dollars)"]) > MEDIAN_INCOME_SUPPRESSION_THRESHOLD].index
housing.loc[medianIncomeSuppressionIndex, [
    'Median income (2023 dollars)', 
    'Median income (year-of dollars)',
    'Median sale price/income ratio (MSA med inc)',
    'Median sale price/income ratio (geography med inc)']] = None
print("Suppressed {} records where median income MOE was above the suppression threshold".format(len(medianIncomeSuppressionIndex)))

Suppress records where the mean income margin of error is more than the specified threshold.

In [ ]:
meanIncomeSuppressionIndex = housing.loc[(housing["Mean income MOE (year-of dollars)"]/housing["Mean income (year-of dollars)"]) > MEAN_INCOME_SUPPRESSION_THRESHOLD].index
housing.loc[meanIncomeSuppressionIndex, [
    'Mean income (2023 dollars)', 
    'Mean income (year-of dollars)']] = None
print("Suppressed {} records where mean income MOE was above the suppression threshold".format(len(meanIncomeSuppressionIndex)))

Load the output schema and verify that the data contains only the columns present in the output schema.

In [ ]:
housingCostSchema = morpc.frictionless_load_schema(HOUSING_COST_SCHEMA_PATH)
housing = housing.filter(items=housingCostSchema.field_names, axis="columns")

Inspect the data.

In [ ]:
housing.head()

## Export data

In [ ]:
housing.to_csv(HOUSING_COST_TABLE_PATH, index=False)

## Create resource file for exported data

In [ ]:
housingResource = morpc.frictionless_create_resource(HOUSING_COST_FILENAME, 
    resourcePath=HOUSING_COST_RESOURCE_PATH,
    title="MORPC Insights | Housing Cost and Income", 
    name="housingcost", 
    description="Home sale price and household income by year for geographies in the MORPC 15-County region.  Sale price data is summarized from Multiple Listing Service (MLS) data.  Income data is from the American Community Survey 1-year estimates.",
    writeResource=True,
    validate=True
)

## Generate static charts

In [ ]:
for f in os.scandir(chartDir):
    os.remove(f)

%matplotlib agg
# Create a list to accumulate geographies for which a thumbnail is generated
platformIncludeList = []
# Iterate over each geography in data set
for geoid in housing["GEOIDFQ"].unique():
    # If the geography is not a MORPC member, skip it. The platform only features members.
    if(not geoid in memberList):
        continue
    
    # Extract the data for a single geography
    temp = housing.loc[housing["GEOIDFQ"] == geoid].copy()

    temp = temp.dropna(subset="Median sale price/income ratio (MSA med inc)")

    if(temp.shape[0] < 2):
        continue
        
    platformIncludeList.append(geoid)

    # Generate a title string
    geoName = temp.iloc[0]["NAME"]
    title = "Home sale price to income ratio - {}".format(geoName)

    # Drop the geography name and type
    temp = temp.filter(items=["YEAR","Median sale price/income ratio (MSA med inc)"], axis="columns")

    # Make the variable names nicer looking
    temp = temp.rename(columns={
        "YEAR":"Year",
        "Median sale price/income ratio (MSA med inc)":"Price-to-income ratio"
    })
    
    yMax = temp["Price-to-income ratio"].dropna().max()
    yLimMax = math.ceil(1.1*yMax)
   
    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))
    
    temp.plot(ax=ax, x="Year", y="Price-to-income ratio", color="black", linewidth="4", marker="o", 
              markersize=8, stacked=True, legend=False)
    ax.set_title(title, fontsize=14)
    ax.set_ylim(ymin=0, ymax=yLimMax)
    ax.set_xlim(xmin=YEAR_RANGE[0], xmax=YEAR_RANGE[1]+1)
    xlabel = None
    ylabel = None
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xticks([round(tick,0) for tick in ax.get_xticks()])
    ax.set_yticks([round(tick,0) for tick in ax.get_yticks()])
    ax.grid(visible=True, color="lightgrey")
    ax.set_axisbelow(True)

    # Everything in the following block is needed to put colored bands on the chart representing afforability thresholds. In matplotlib, this is straightforward
    # See the ax.fill_between lines.  In Excel ossible using the xlsxwriter.Chart.set_plotarea() function with a gradient, but it is a bit of a pain.  Gradient
    # stops need to be specified on a scale of 0 to 100, so they need to be computed separately for each chart.
    # First define the colors (muted green) and the breakpoints for the "Affordable' band    
    gradientColors = ['#80C080','#80C080']
    if(yLimMax < AFFORDABILITY_BREAKPOINTS['Moderately Unaffordable']):
        # If the highest value is still lower than the next affordability band, set the top gradient stop to max scale.
        gradientPositions = [0, 100]
    else:
        # If the highest value is above the next affordability band, set the top gradient stop to the top of the current band.
        gradientPositions = [0, AFFORDABILITY_BREAKPOINTS['Moderately Unaffordable']/yLimMax*100] 
    # Generate the color band on the matplotlib chart
    ax.fill_between(range(int(ax.get_xlim()[0]), int(ax.get_xlim()[1]+1)), 0, min(AFFORDABILITY_BREAKPOINTS['Moderately Unaffordable'], yLimMax), facecolor='green', alpha=0.5)
    if(yLimMax > AFFORDABILITY_BREAKPOINTS['Moderately Unaffordable']):
        # Define the colors (muted yellow) and the breakpoints for the "Moderately Unaffordable" band
        gradientColors = gradientColors + ['#FFFF80','#FFFF80']
        if(yLimMax < AFFORDABILITY_BREAKPOINTS['Seriously Unaffordable']):
            # If the highest value is still lower than the next affordability band, set the top gradient stop to max scale. Use the upper stop of the previous band as the
            # lower stop for this band.
            gradientPositions = gradientPositions + [AFFORDABILITY_BREAKPOINTS['Moderately Unaffordable']/yLimMax*100, 100]    
        else:
            # If the highest value is above the next affordability band, set the top gradient stop to the top of the current band.
            gradientPositions = gradientPositions + [AFFORDABILITY_BREAKPOINTS['Moderately Unaffordable']/yLimMax*100, AFFORDABILITY_BREAKPOINTS['Seriously Unaffordable']/yLimMax*100]    
        ax.fill_between(range(int(ax.get_xlim()[0]), int(ax.get_xlim()[1]+1)), AFFORDABILITY_BREAKPOINTS['Moderately Unaffordable'], min(AFFORDABILITY_BREAKPOINTS['Seriously Unaffordable'], yLimMax), facecolor='yellow', alpha=0.5)
    if(yLimMax > AFFORDABILITY_BREAKPOINTS['Seriously Unaffordable']):
        gradientColors = gradientColors + ['#FFD280','#FFD280']
        if(yLimMax < AFFORDABILITY_BREAKPOINTS['Severely Unaffordable']):
            gradientPositions = gradientPositions + [AFFORDABILITY_BREAKPOINTS['Seriously Unaffordable']/yLimMax*100, 100]    
        else:
            gradientPositions = gradientPositions + [AFFORDABILITY_BREAKPOINTS['Seriously Unaffordable']/yLimMax*100, AFFORDABILITY_BREAKPOINTS['Severely Unaffordable']/yLimMax*100]  
        ax.fill_between(range(int(ax.get_xlim()[0]), int(ax.get_xlim()[1]+1)), AFFORDABILITY_BREAKPOINTS['Seriously Unaffordable'], min(AFFORDABILITY_BREAKPOINTS['Severely Unaffordable'], yLimMax), facecolor='orange', alpha=0.5)
    if(yLimMax > AFFORDABILITY_BREAKPOINTS['Severely Unaffordable']):
        gradientColors = gradientColors + ['#FF8080','#FF8080']
        gradientPositions = gradientPositions + [AFFORDABILITY_BREAKPOINTS['Severely Unaffordable']/yLimMax*100, 100]
        ax.fill_between(range(int(ax.get_xlim()[0]), int(ax.get_xlim()[1]+1)), AFFORDABILITY_BREAKPOINTS['Severely Unaffordable'], yLimMax, facecolor='red', alpha=0.5)    
   
    # Format the y-axis labels as integers with comma separators
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "{}.svg".format(geoid)))

    plt.close(ax.figure)
    
    writer = pd.ExcelWriter(os.path.join(chartDir, "{}.xlsx".format(geoid)), engine='xlsxwriter')
    dataOptions = {
        "numberFormat": "#,##0.0",
        "columnWidth": 20
    }
    chartOptions = {
        "colors": "black",
        "titles": {
            "chartTitle": title,
            "xTitle": xlabel,
            "yTitle": ylabel
        },
        "hideLegend": True,
        "seriesOptions": [{
            "line": {"width":3},
            "marker": {"type":"circle", "size":7}
        }],
        "xAxisOptions": {
            "num_font": {"size":16},
            "min": YEAR_RANGE[0],
            "max": YEAR_RANGE[1]+1
        },
        "yAxisOptions": {
            "min": 0, 
            "max": yLimMax,
            "num_font": {"size":16},
            "num_format": "0",
            "major_unit": 1
        },
        "plotAreaOptions": {
            'gradient': {
                'colors': gradientColors,
                'positions': gradientPositions,
                'type': "linear",
                'angle': 270
            }            
        },
        "sizeOptions": {
            "x_scale": 1.5,
            "y_scale": 1.5
        }
    }
    morpc.data_chart_to_excel(temp.set_index("Year"), writer, chartType="line", dataOptions=dataOptions, chartOptions=chartOptions)
    writer.close()
    

%matplotlib inline

## Generate Insights catalog content

The content in the Insights platform is controlled by a catalog spreadsheet. Each tile to be displayed in the platform must have a record in the catalog.  This section will create the records for the tiles that display the alternative fuel station data.  Eventually this function will be performed by a separate staging script.

First specify the column names used in the catalog.

In [ ]:
columnNames=["TileID","TilesetID","GeographyType","GeographyName","Category","Headline","Commentary","ThumbnailURL","Contributor","Vintage","UpdateInterval","ShareURL","DataProductURL","MoreInformationURL"]

Create a new dataframe containing only the geographies for which thumbnail images were produced in the section above.

In [ ]:
catalog = housing.loc[housing["GEOIDFQ"].isin(platformIncludeList)].copy()

Extract only the metadata columns of interest and flatten the data to have only one record per geography. Rename the metadata fields to match the catalog fields.

In [ ]:
catalog = catalog.filter(items=["GEOIDFQ","NAME","GEOTYPE"], axis="columns") \
    .groupby("GEOIDFQ").first() \
    .reset_index() \
    .rename(columns={"NAME":"GeographyName","GEOTYPE":"GeographyType"})

Change the GeographyType values to match the schema of the catalog.

In [ ]:
catalog["GeographyType"] = catalog["GeographyType"].map({
    "REGION15":"Region",
    "COUNTY":"County",
    "COUNTY-TOWNSHIP-REMAINDER":"Community",
    "PLACE":"Community"
})

Populate some placeholder fields.

In [ ]:
catalog["TileID"] = None
catalog["TilesetID"] = None
catalog["Category"] = None
catalog["Headline"] = "TBD"
catalog["Commentary"] = "TBD"

Generate the URL for the thumbnail images. These will be hosted in GitHub and will be indexed by GEOIDFQ.

In [ ]:
catalog["ThumbnailURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://raw.githubusercontent.com/morpc-insights/insights-housingprice/refs/heads/main/output_data/charts/{}.svg".format(geoid))

Populate some other simple metadata.  Vintage in this case refers to the year that the content was published in Insights. This is to give readers an idea of how old it is.  UpdateInterval gives them an idea of when to expect the next version. ShareURL is a placeholder for now.

In [ ]:
catalog["Contributor"] = "Mid-Ohio Regional Planning Commission"
catalog["Vintage"] = str(datetime.date.today().year)
catalog["UpdateInterval"] = "annually"
catalog["ShareURL"] = None

Generate the data product URL.  This points to an ArcGIS Dashboard that accepts URL parameters.  GEOIDFQ is passed as a parameter to tell the app to load the data for a particular geography.

In [ ]:
catalog["DataProductURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://www.arcgis.com/apps/dashboards/237dd7c3dc694feebe0b3e6381f45f93#geoid={}".format(geoid))

Generate the URLs that point to the extended commentary pages.  Default to a common page (population.pdf) hosted in GitHub.  Point to specific pages for the 15-county region and for each county.

In [ ]:
catalog["MoreInformationURL"] = "https://morpc1-my.sharepoint.com/:w:/g/personal/aporr_morpc_org/EeowBvR-mvJEsEDFbiSWb1kBE5DcjowKjyYzzKE35vwfwg?e=KCvIsQ"

Extract only the required columns.

In [ ]:
catalog = catalog.filter(items=columnNames, axis="columns")

Inspect the listing.

In [ ]:
catalog.head()

Save the catalog to an Excel spreadsheet.

In [ ]:
catalog.to_excel("catalog.xlsx", index=False)

It is necessary to manually add these records to the master catalog or update the records already therein.  See the following file in GitHub:

https://github.com/morpc/morpc-insights/blob/main/catalog/morpc_insights_catalog.xlsx
